In [4]:
#protein solubility prediction
#interactive inline plot
%matplotlib notebook 

#Import all the packages
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer
from sklearn import svm

# Importing pathlib package to allow parsing of csv files
from pathlib import Path
import sys

In [5]:

cleandata_1name = Path(r"cleandata_1name.csv").parent.absolute() / Path("cleandata_1name.csv") # Making PATH object of clean14w.csv
totalCGAN7_file = Path(r"totalCGAN7.csv").parent.absolute() / Path("totalCGAN7.csv") # Making PATH object of totalCGAN7.csv
print(cleandata_1name)
print(cleandata_1name.exists())

c:\Users\erena\Google Drive\Canada\School\Summer 2020\Mahadevan Lab\Protein Solubility Paper\Python Files\cleandata_1name.csv
True


In [6]:
#first defind the headers of the columns (the different proteins) and then load data into dataframe
headers = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V','solubility']

# Need to make a PATH object to pass to pd.read_cv for "clean14w.csv" and "totalCGGAN7.csv"

#original data
df_Original = pd.read_csv(cleandata_1name,index_col=0,names=headers,header=0) # Should be clean14w.csv
df_Original.dropna(axis=0,inplace=True)
#gan supplemented data
df_WithGan = pd.read_csv(cleandata_1name,index_col=0,names=headers,header=0) # should be totalCGAN7.csv
df_WithGan.dropna(axis=0,inplace=True)

In [2]:
def Neural_Network_Prediciton(df):
    #set the X(proteins) and y(soubility)
    X = df.iloc[:,0:-1]; # iloc = integer locate/selection by position
    y = df.solubility;

    X = X.to_numpy().astype(np.float)
    y = y.to_numpy().astype(np.float)

    #set 10 fold validation
    kf = KFold(n_splits=10)

    def mc_reg(y_true, y_pred):
        y_true[y_true >= 0.44] = True; y_true[y_true < 0.44] = False
        y_pred[y_pred >= 0.44] = True; y_pred[y_pred < 0.44] = False
        return matthews_corrcoef(y_true, y_pred) # returns matthews corrcoeff with same y_true and y_pred           calculated for all three of mc, accracy, precision
    
    def accracy_reg(y_true, y_pred):
        y_true[y_true >= 0.44] = True; y_true[y_true < 0.44] = False
        y_pred[y_pred >= 0.44] = True; y_pred[y_pred < 0.44] = False
        return accuracy_score(y_true, y_pred)
    
    def precision_reg(y_true,y_pred):
        y_true[y_true >= 0.44] = True; y_true[y_true < 0.44] = False
        y_pred[y_pred >= 0.44] = True; y_pred[y_pred < 0.44] = False
        return average_precision_score(y_true, y_pred)

    #use the best results for prediction
    reg = MLPRegressor(hidden_layer_sizes=(31, ),activation='relu', learning_rate='invscaling',learning_rate_init=0.003)

    method_original = {'accuracy':make_scorer(accracy_reg),'precision':make_scorer(precision_reg),'AUC':'roc_auc','MCC':make_scorer(mc_reg),'r2':'r2'}

    results_original = cross_validate(reg, X, y, cv=kf, scoring=method_original, return_train_score=False)
    
    #out put the scores
    score_original = pd.DataFrame([[np.mean(results_original['test_accuracy']),np.mean(results_original['test_precision']),
                                    np.mean(results_original['test_AUC']),np.mean(results_original['test_MCC']),
                                    np.mean(results_original['test_r2'])]],
                                    columns=['accuracy','precision','AUC','MCC','R2'])
    
    #output the predictions
    # create an empty dictionary for output
    predictions = {}
    index = 0
    for train, test in kf.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        reg.fit(X_train,y_train)
        y_predict = reg.predict(X_test)
        predictions[index] = pd.DataFrame(np.column_stack((predicted_original,y_original)),columns=['Predicted','Original'])
        index+=1
        
    return score_original,predictions

#call the evaluation function
score_original,predictions = Neural_Network_Prediciton(df_Original)

#print score
score_original

In [7]:
[result_original, predictions] = Neural_Network_Prediciton(df_WithGan) # added predictions
result_original

NameError: name 'predicted_original' is not defined

In [ ]:
#input 0-9 to retrieve the different folds
predictions[9]
